# Utils

> Utility functions


In [ ]:
#| default_exp utils

In [ ]:
#| hide
from nbdev.showdoc import *

from IPython.display import clear_output, DisplayHandle

def update_patch(self, obj):
    clear_output(wait=True)
    self.display(obj)
DisplayHandle.update = update_patch

In [ ]:
#| export
from fastai.vision.all import torch, nn
import numpy as np
from torchmetrics.functional.image import structural_similarity_index_measure as structural_similarity
from torchmetrics.functional.image import peak_signal_noise_ratio
import math

/home/bm/miniconda3/envs/n2m/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#| export
def attributesFromDict(d):
    self = d.pop('self')
    for n, v in d.items():
        setattr(self, n, v)

In [ ]:
#| export

def gaussian_diag(mean, logsd):
    class o(object):
        pass

    o.mean = mean
    o.logsd = logsd
    o.eps = torch.normal(torch.zeros(mean.shape, device=mean.device), torch.ones(mean.shape, device=mean.device))
    o.sample = mean + torch.exp(logsd) * o.eps
    o.sample2 = lambda eps: mean + torch.exp(logsd) * eps

    o.logps = lambda x: -0.5 * (np.log(2 * np.pi) + 2. * o.logsd + (x - o.mean) ** 2 / torch.exp(2. * o.logsd))
    o.logp = lambda x: torch.sum(o.logps(x), dim=[1, 2, 3])
    o.get_eps = lambda x: (x - mean) / torch.exp(logsd)
    return o

In [ ]:
#| export

def batch_PSNR(img, imclean, data_range):
    Img = img.data.cpu().numpy().astype(np.float32)
    Iclean = imclean.data.cpu().numpy().astype(np.float32)
    PSNR = 0
    for i in range(Img.shape[0]):
        PSNR += peak_signal_noise_ratio(Iclean[i,:,:,:], Img[i,:,:,:], data_range=data_range)
    return (PSNR/Img.shape[0])

### Train utils


In [ ]:
#| export

def weights_init_kaiming(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.kaiming_normal_(m.weight.data, a=0, mode='fan_in')
    elif classname.find('Linear') != -1:
        nn.init.kaiming_normal_(m.weight.data, a=0, mode='fan_in')
    elif classname.find('BatchNorm') != -1:
        # nn.init.uniform(m.weight.data, 1.0, 0.02)
        m.weight.data.normal_(mean=0, std=math.sqrt(2./9./64.)).clamp_(-0.025,0.025)
        nn.init.constant_(m.bias.data, 0.0)


In [ ]:
#| export
def output_psnr_mse(img_orig, img_out, data_range):
    squared_error = torch.square(img_orig - img_out)
    mse = torch.mean(squared_error)
    psnr = 10 * torch.log10(data_range ** 2/ mse)
    return psnr

def mean_psnr(denoised, imclean, data_range=1.0):
    n_blk = imclean.shape[0]
    mean_psnr = 0
    psnrs = np.ndarray([n_blk])
    for b in range(n_blk):
        ref_block = imclean[b]
        res_block = denoised[b]
        psnr = output_psnr_mse(ref_block, res_block, data_range)
        mean_psnr += psnr
        psnrs[b] = psnr
    return mean_psnr / n_blk, psnrs


def kldiv_simple(real_noise, sampled_noise):
    bw = 0.2 / 64	
    bin_edges = np.concatenate(([-1000.0], np.arange(-0.1, 0.1 + 1e-9, bw), [1000.0]), axis=0)
    cnt_regr = 1	
    real_hist = get_histogram(real_noise, bin_edges=bin_edges, cnt_regr=cnt_regr)
    sampled_hist = get_histogram(sampled_noise, bin_edges=bin_edges, cnt_regr=cnt_regr)

    kld = kl_div_forward(real_hist, sampled_hist)

    return kld

def get_histogram(data, bin_edges=None, cnt_regr=1):
    n = np.prod(data.shape)	
    hist, _ = np.histogram(data, bin_edges)	
    return (hist + cnt_regr)/(n + cnt_regr * len(hist))

def kl_div_forward(p, q):
    assert (~(np.isnan(p) | np.isinf(p) | np.isnan(q) | np.isinf(q))).all()	
    idx = (p > 0)
    p = p[idx]
    q = q[idx]
    return np.sum(p * np.log(p / q))

def batch_SSIM(img, imclean, data_range):
    Img = img.data.cpu().numpy().astype(np.float32)
    Iclean = imclean.data.cpu().numpy().astype(np.float32)
    SSIM = 0
    for i in range(Img.shape[0]):
        SSIM += structural_similarity(Iclean[i].transpose((1, 2, 0)), Img[i].transpose((1, 2, 0)), data_range=data_range,  sigma=0.8)
    return (SSIM/Img.shape[0])

def weights_init_orthogonal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.orthogonal_(m.weight.data)
    elif classname.find('Linear') != -1:
        nn.init.orthogonal_(m.weight.data)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(mean=0, std=math.sqrt(2./9./64.)).clamp_(-0.025,0.025)
        nn.init.constant_(m.bias.data, 0.0)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()